# Load Stuff

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
classes = {
    0: ("actinic keratoses and intraepithelial carcinomae(Cancer)"),
    1: ("basal cell carcinoma(Cancer)"),
    2: ("benign keratosis-like lesions(Non-Cancerous)"),
    3: ("dermatofibroma(Non-Cancerous)"),
    4: ("melanocytic nevi(Non-Cancerous)"),
    5: ("pyogenic granulomas and hemorrhage(Can lead to cancer)"),
    6: ("melanoma(Cancer)"),
}

In [3]:
# Load train and test set into X and Y
train_path = "dataset\hmnist_28_28_RGB_train.csv"
test_path = "dataset\hmnist_28_28_RGB_test.csv"

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

y_train = train_set['label']
x_train = train_set.drop(columns=['label'])
x_train=np.array(x_train).reshape(-1,28,28,3)

y_test = test_set['label']
x_test = test_set.drop(columns=['label'])
x_test=np.array(x_test).reshape(-1,28,28,3)


In [8]:
# Load the pre-trained model
model = keras.models.load_model('best_model.h5')
model.trainable = True

In [9]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 14, 14, 16)        64        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 32)        4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0

In [10]:
# Accuracy of pre-trained model on test set
score = model.evaluate(x_test, y_test, verbose=0, batch_size=1)
print('Test loss:', score[0])
print('Test accuracy (%):', 100*score[1])

Test loss: 1.4607079029083252
Test accuracy (%): 72.54118919372559


# Attack

### Function to Make Pertubated Images 

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

def create_compromised_image(image, epsilon):
  image = tf.convert_to_tensor(image.reshape((1, 28, 28, 3)), dtype=tf.float32)

  with tf.GradientTape() as tape:
    tape.watch(image)
    prediction = model(image)
    label = tf.argmax(prediction, axis=1)
    loss = loss_object(label, prediction)

  gradient = tape.gradient(loss, image)
  perturbations = tf.sign(gradient)

  adversarial_image = image + epsilon*perturbations
  return adversarial_image


### Test Out Attack

In [ ]:
def display_images(new_image, description):
  #  Display the given image along with its predicted label and confidence.
  pred = model.predict(new_image)
  label = np.argmax(pred, axis=1)[0]
  label = classes[label]
  confidence = np.max(pred, axis=1)[0]
  plt.figure()
  plt.imshow(new_image[0]*0.5/255+0.5)
  plt.title('{} \n {} : {:.2f}% Confidence'.format(description, label, confidence*100))
  plt.show()

epsilons = np.array([0, 0.005, 0.01])*255
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input')
                for eps in epsilons]

for i, eps in enumerate(epsilons):
  adv_x = create_compromised_image(x_test[0], eps)
  display_images(adv_x, descriptions[i])

### Test Attck on multiple images and plot the accuracy for different epsilons

In [ ]:
epsilon = np.linspace(0, 245, 20)

length = 20

test_loss = []
test_acc = []

for eps in epsilon:
    x_test_compromised = []
    for i, img in enumerate(x_test[:length]):
        adv_x = create_compromised_image(img, eps)[0].numpy()
        x_test_compromised.append(adv_x)

    x_test_compromised = np.array(x_test_compromised)
    score = model.evaluate(x_test_compromised, y_test[:length], verbose=0, batch_size=1)
    test_loss.append(score[0])
    test_acc.append(score[1])

plt.figure()
plt.plot(epsilon, test_loss)
plt.title('Loss vs Epsilon')
plt.xlabel('Epsilon')
plt.ylabel('Loss')
plt.show()

plt.figure()
plt.plot(epsilon, test_acc)
plt.title('Accuracy vs Epsilon')
plt.xlabel('Epsilon')
plt.ylabel('Accuracy')
plt.show()

### Make Equivalent CSV file of Pertubated Images

In [ ]:
# Load all images
comp_path = 'dataset\\hmnist_28_28_RGB_0e055.csv'

def make_and_store_pertubated_images(og_path, comp_path, epsilon):

    og_df=pd.read_csv(og_path)

    labels = og_df['label']
    og_img = og_df.drop(columns=['label'])
    og_img=np.array(og_img).reshape(-1,28,28,3)

    # Create compromised dataset
    compromised_img = []
    epsilon = 0.055
    for i, img in enumerate(og_img):
        adv_x = create_compromised_image(img, epsilon*255).numpy()
        compromised_img.append(adv_x)
    compromised_img = np.array(compromised_img)

    # Evaluate compromised dataset
    score = model.evaluate(compromised_img, labels, verbose=0, batch_size=1)
    print('Test loss:', score[0])
    print('Test accuracy (%):', 100*score[1])

    # Save as csv
    compromised_img = compromised_img.reshape(-1, 28*28*3)
    compromised_img_with_labels = np.concatenate((compromised_img, np.array(labels)[:, None]), axis=1)
    comp_df = pd.DataFrame(compromised_img_with_labels)
    comp_df.columns = og_df.columns
    comp_df.to_csv(comp_path, index=False)

# make_and_store_pertubated_images(og_path, comp_path, 0.055)

# Defence

### Load Pertubated Images and Evaluate

In [ ]:
comp_df=pd.read_csv(comp_path)

comp_labels = comp_df['label']
comp_img = comp_df.drop(columns=['label'])
comp_img=np.array(comp_img).reshape(-1,28,28,3)

# Evaluate compromised dataset
score = model.evaluate(comp_img, comp_labels, verbose=0, batch_size=1)
print('Test loss:', score[0])
print('Test accuracy (%):', 100*score[1])

### Mix Pertubated Images With Normal images

In [ ]:
og_df = pd.read_csv(og_path)

mixed_df = pd.concat([og_df, comp_df], ignore_index=True)
mixed_df = mixed_df.sample(frac=1)

